In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "4"

In [2]:
%load_ext autoreload
%autoreload 2
%cd code

C:\Users\maja9\Documents\GitHub\Clustering-Algorithms\code


In [3]:
from sklearn.metrics import adjusted_rand_score, silhouette_score
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score

def adjusted_rand_index(labels_true, labels_pred):
    return adjusted_rand_score(labels_true, labels_pred)

def purity_score(y_true, y_pred):
    # Compute contingency matrix
    contingency_matrix = pd.crosstab(y_true, y_pred)
    # Sum of maximum values in each column
    return np.sum(np.amax(contingency_matrix.values, axis=0)) / np.sum(contingency_matrix.values)

def davies_bouldin_index(data, labels):
    return davies_bouldin_score(data, labels)

def silhouette_coefficient(data, labels):
    return silhouette_score(data, labels)

def f_measure(labels_true, labels_pred):
    contingency_matrix = pd.crosstab(labels_true, labels_pred)
    precision = contingency_matrix.max(axis=0).sum() / len(labels_pred)
    recall = contingency_matrix.max(axis=1).sum() / len(labels_true)
    return 2 * (precision * recall) / (precision + recall)

def calinski_harabasz_index(data, labels):
    return calinski_harabasz_score(data, labels)


def get_metrics_general(X, labels_true, labels_pred, method, time, n_iterations = None):
    # Compute metrics
    dbi = davies_bouldin_index(X, labels_pred)
    silhouette = silhouette_coefficient(X, labels_pred)
    calinski = calinski_harabasz_index(X, labels_pred)

    ari = adjusted_rand_index(labels_true, labels_pred)
    purity = purity_score(labels_true, labels_pred)
    fmeasure = f_measure(labels_true, labels_pred)

    # Append results
    results = {
        "Method": method,
        "ARI": ari,
        "Purity": purity,
        "F-Measure": fmeasure,
        "Davies-Bouldin Index": dbi,
        "Silhouette Coefficient": silhouette,
        "Calinski": calinski,
        "Solving Time": time,
        "Iterations": n_iterations if n_iterations else np.NaN
    }
    return results


In [4]:
import time
import pandas as pd
import numpy as np
from sklearn.cluster import SpectralClustering

def run_all_spectral_clustering(data_X, data_y):
    results = []

    for k in range(2, 14):
        for affinity in ["rbf", "nearest_neighbors"]:
            for assign_labels in ['kmeans', 'discretize']:
                for eigen_solver in ['arpack', 'lobpcg']:

                    if affinity == "nearest_neighbors":
                        for n_neighbors in [25, 50, 100, 200]:

                            start_time = time.time()
                            spectral = SpectralClustering(
                                n_clusters=k,
                                affinity=affinity,
                                assign_labels=assign_labels,
                                eigen_solver=eigen_solver,
                                n_neighbors=n_neighbors,
                            )

                            labels = spectral.fit_predict(data_X)

                            end_time = time.time()
                            elapsed_time = end_time - start_time

                            method= f"spectral_k{k}_affinity{affinity.replace('_', '-')}_n-neighbors{n_neighbors}_assign-labels{assign_labels.replace('_', '-')}_eigen-solver{eigen_solver.replace('_', '-')}"

                            results_spectral = get_metrics_general(data_X, data_y, labels, method, elapsed_time,n_iterations = None)

                            results.append(results_spectral)

                    else:
                        start_time = time.time()
                        spectral = SpectralClustering(
                            n_clusters=k,
                            affinity=affinity,
                            assign_labels=assign_labels,
                            eigen_solver=eigen_solver,
                        )

                        labels = spectral.fit_predict(data_X)

                        end_time = time.time()
                        elapsed_time = end_time - start_time

                        method= f"spectral_k{k}_affinity{affinity.replace('_', '-')}_assign-labels{assign_labels.replace('_', '-')}_eigen-solver{eigen_solver.replace('_', '-')}"

                        results_spectral = get_metrics_general(data_X, data_y, labels, method, elapsed_time,n_iterations = None)

                        results.append(results_spectral)

    results_df = pd.DataFrame(results)
    return results_df

In [5]:
df = pd.read_csv('../datasets_processed/grid.csv')
df_X = np.array(df[df.columns[:-1]])
df_y = np.array(df[df.columns[-1]])

resultados = run_all_spectral_clustering(df_X, df_y)

resultados.to_csv('../output/results_spectral_grid.csv')

In [6]:
df = pd.read_csv('../datasets_processed/sick.csv')
df_X = np.array(df[df.columns[:-1]])
df_y = np.array(df[df.columns[-1]])

resultados = run_all_spectral_clustering(df_X, df_y)

resultados.to_csv('../output/results_spectral_sick.csv')

C:\Users\maja9\anaconda3\envs\Clustering-Algorithms\lib\site-packages\sklearn\manifold\_spectral_embedding.py:374: UserWarning: Exited postprocessing with accuracies 
[1.01333267e-15 1.52005723e-06 1.61161478e-06 1.52225878e-06
 1.05039106e-06 2.10754880e-06 5.74449927e-06 9.06283294e-06
 1.00677806e-05]
not reaching the requested tolerance 1e-05.
  _, diffusion_map = lobpcg(


In [7]:
df = pd.read_csv('../datasets_processed/vowel.csv')
df_X = np.array(df[df.columns[:-1]])
df_y = np.array(df[df.columns[-1]])

resultados = run_all_spectral_clustering(df_X, df_y)

resultados.to_csv('../output/results_spectral_vowel.csv')

C:\Users\maja9\anaconda3\envs\Clustering-Algorithms\lib\site-packages\sklearn\manifold\_spectral_embedding.py:259: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
C:\Users\maja9\anaconda3\envs\Clustering-Algorithms\lib\site-packages\sklearn\manifold\_spectral_embedding.py:259: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
C:\Users\maja9\anaconda3\envs\Clustering-Algorithms\lib\site-packages\sklearn\manifold\_spectral_embedding.py:259: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
C:\Users\maja9\anaconda3\envs\Clustering-Algorithms\lib\site-packages\sklearn\manifold\_spectral_embedding.py:259: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
C:\Users\maja9\anaconda3\envs\Clustering-Algorithms\lib\site-packages\sklearn\manifold\_spectral_embedding.py:259: UserW